# DeltaCNN Installation

To be able to import the **deltacnn library**, we first need to clone the [DeltaCNN GitHub repository](https://github.com/facebookresearch/DeltaCNN).

We then need to install the DeltaCNN framework. The README.md recommends to call the setup.py file directly using the following:
```
python setup.py install --user
```
However, we were not successful in installing the framework this way. Rather, we will **install using pip** directly on the DeltaCNN-Main folder.

In [ ]:
!git clone https://github.com/facebookresearch/DeltaCNN.git
!pip install /content/DeltaCNN/ # original


fatal: destination path 'DeltaCNN' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./DeltaCNN
  Preparing metadata (setup.py) ... done
  Created wheel for torchdeltacnn: filename=torchdeltacnn-0.0.0-cp39-cp39-linux_x86_64.whl size=17505296 sha256=fd1a28c83331af360ffe5118a044600157a3fc9d24882f3540d32d03a12fc40a
  Stored in directory: /tmp/pip-ephem-wheel-cache-7r_wmfcj/wheels/79/07/be/b2e778efc1014ae6aaec3e3783a32762efddf8f6437493ea96
Successfully built torchdeltacnn
  Attempting uninstall: torchdeltacnn
    Found existing installation: torchdeltacnn 0.0.0
    Uninstalling torchdeltacnn-0.0.0:
      Successfully uninstalled torchdeltacnn-0.0.0


We can now install the deltacnn package and other modules like torch, numpy, etc.:

In [ ]:
# Setup
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import numpy as np
import torch
import torch.nn as nn
import deltacnn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchsummary import summary
from PIL import Image

import time
import copy

# Additional setup to use Tensorboard
!pip install -q tensorflow
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# MNIST Installation
We load the MNIST dataset using torchvision and perform preprocessing on the dataset:

In [ ]:
# Preprocessing data: convert to tensors and normalize by subtracting dataset
# mean and dividing by std.
torch.manual_seed(0)
pixel_permutation = torch.randperm(28*28)
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

# Get data from torchvision.datasets
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Define data loaders used to iterate through dataset
train_loader = DataLoader(train_data, batch_size= 8, drop_last=True, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 8, drop_last=True)

# Training the Model
Firstly, we setup some functions to compute the accuracy of the model for a given network and dataloader:

In [ ]:
def evaluate_train_accuracy(data_loader, net, device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  #make sure network is in evaluation mode

    #init
    acc_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0] #increases with the number of samples in the batch
    return acc_sum.item()/n

In [ ]:
def evaluate_test_accuracy(data_loader, net, device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  #make sure network is in evaluation mode

    #init
    acc_sum = torch.tensor([0], dtype=torch.float32, device=device)
    n = 0

    for X, y in data_loader:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0] #increases with the number of samples in the batch
    return acc_sum.item()/n

Let's define the standard CNN model:

In [ ]:
class Net(nn.Module):
    """
    3-layer CNN network with max pooling
    
    Args:
        in_channels: number of features of the input image ("depth of image")
        hidden_channels: number of hidden features ("depth of convolved images")
        out_features: number of features in output layer
    """
    
    def __init__(self, in_channels, hidden_channels, out_features):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, hidden_channels[0],
                               kernel_size=3,
                               padding=1)
        self.relu1 = nn.ReLU()
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(hidden_channels[0], hidden_channels[1],
                               kernel_size=5,
                               padding=2)
        self.relu2 = nn.ReLU()
        self.max_pool2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(7*7*hidden_channels[1], out_features)

    def forward(self, x):
        # First convolutional layer
        x = self.conv1(x)
        # Activation function
        x = self.relu1(x)
        # Max pool
        x = self.max_pool1(x)
        # Second convolutional layer
        x = self.conv2(x)
        # Activation function
        x = self.relu2(x)
        # Max pool
        x = self.max_pool2(x)
        # Flatten
        x = x.view(x.size(0), -1)
        # Fully connected layer
        x = self.fc(x)
        return x

Now, let's modify the standard architecture to use the DeltaCNN framework:

In [ ]:
class DCNet(deltacnn.DCModule):
    """
    3-layer CNN network with max pooling
    
    Args:
        in_channels: number of features of the input image ("depth of image")
        hidden_channels: number of hidden features ("depth of convolved images")
        out_features: number of features in output layer
    """
    
    def __init__(self, in_channels, hidden_channels, out_features):
        super(DCNet, self).__init__()

        self.conv1 = deltacnn.DCConv2d(in_channels, hidden_channels[0],
                               kernel_size=3,
                               padding=1)
        self.relu1 = deltacnn.DCActivation(activation="relu")
        self.max_pool1 = deltacnn.DCMaxPooling(2)
        self.conv2 = deltacnn.DCConv2d(hidden_channels[0], hidden_channels[1],
                               kernel_size=5,
                               padding=2)
        self.relu2 = deltacnn.DCActivation(activation="relu")
        self.max_pool2 = deltacnn.DCMaxPooling(2)
        self.fc = nn.Linear(4056, out_features)

        self.sparsify = deltacnn.DCSparsify()
        self.densify = deltacnn.DCDensify()

    def forward(self, x):
        # Sparsify input
        x = self.sparsify(x) 
        # First convolutional layer
        x = self.conv1(x)
        # Activation function
        x = self.relu1(x)
        # Max pool
        x = self.max_pool1(x)
        # Second convolutional layer
        x = self.conv2(x)
        # Activation function
        x = self.relu2(x)
        # Max pool
        x = self.max_pool2(x)
        # Densify output
        x = self.densify(x)
        # Flatten
        x = x.reshape(x.size(0), -1)
        # Fully connected layer
        x = self.fc(x)
        return x


We can now train the selected model:

In [ ]:
# Create a writer to write to Tensorboard
writer = SummaryWriter()

in_channels = 1 # Black-white images in MNIST digits
hidden_channels = [5, 6]
out_features = 10 

# Training parameters
learning_rate = 0.0015
momentum = 0.9
weight_decay = 0.001
dampening = 0.2
epochs = 10

# Try using gpu instead of cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize network
net = DCNet(in_channels, hidden_channels, out_features) # select which model to train
net.to(device, memory_format=torch.channels_last) # set the network in channels last mode
net.process_filters() # convert filters into DeltaCNN format
optimizer = torch.optim.SGD(net.parameters(), lr = learning_rate, momentum = momentum, weight_decay = weight_decay , dampening = dampening)
criterion = nn.CrossEntropyLoss()

# Define list to store losses and performances of each iteration
train_losses = []
train_accs = []
test_accs = []

overall_start_time = time.time()
for epoch in range(epochs):
    epoch_start_time = time.time()

    # Network in training mode and to device
    net.train()
    net.to(device)

    # Training loop
    for i, (x_batch, y_batch) in enumerate(train_loader):

        # Set to same device
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        # Set the gradients to zero
        optimizer.zero_grad()

        # Perform forward pass
        y_pred = net(x_batch)

        # Compute the loss
        loss = criterion(y_pred, y_batch)
        train_losses.append(loss)
        
        # Backward computation and update
        loss.backward()
        optimizer.step()

    # Compute train and test error
    train_acc = 100*evaluate_train_accuracy(train_loader, net, device)
    test_acc = 100*evaluate_test_accuracy(test_loader, net, device)

    epoch_time = time.time() - epoch_start_time
    writer.add_scalars('Accuracy', {'Train': train_acc, 'Test': test_acc}, epoch)
    writer.add_scalars('Time', {'Train': epoch_time}, epoch)
    
    # Development of performance
    train_accs.append(train_acc)
    test_accs.append(test_acc)

    # Print performance
    print('Epoch: {:.0f}'.format(epoch+1))
    print('Accuracy of train set: {:.2f}%'.format(train_acc))
    print('Accuracy of test set: {:.2f}%'.format(test_acc))
    print('')

overall_time = time.time() - overall_start_time
print('Overall time: {:.2f}'.format(overall_time))
writer.flush()
writer.close()


Epoch: 1
Accuracy of train set: 93.65%
Accuracy of test set: 94.24%

Epoch: 2
Accuracy of train set: 95.07%
Accuracy of test set: 95.15%

Epoch: 3
Accuracy of train set: 95.49%
Accuracy of test set: 95.58%

Epoch: 4
Accuracy of train set: 95.90%
Accuracy of test set: 95.84%

Epoch: 5
Accuracy of train set: 96.20%
Accuracy of test set: 96.16%

Epoch: 6
Accuracy of train set: 96.30%
Accuracy of test set: 96.09%

Epoch: 7
Accuracy of train set: 96.32%
Accuracy of test set: 96.29%

Epoch: 8
Accuracy of train set: 96.39%
Accuracy of test set: 96.24%

Epoch: 9
Accuracy of train set: 96.62%
Accuracy of test set: 96.41%

Epoch: 10
Accuracy of train set: 96.68%
Accuracy of test set: 96.52%

Epoch: 11
Accuracy of train set: 96.62%
Accuracy of test set: 96.56%

Epoch: 12
Accuracy of train set: 96.77%
Accuracy of test set: 96.55%

Epoch: 13
Accuracy of train set: 96.88%
Accuracy of test set: 96.56%

Epoch: 14
Accuracy of train set: 96.78%
Accuracy of test set: 96.77%

Epoch: 15
Accuracy of train s

KeyboardInterrupt: ignored

Draw the learning curves:

In [ ]:
train_losses = [t.cpu().detach().numpy() for t in train_losses]

# Plot training curves
plt.figure(figsize=(9,4))
plt.subplot(1,2,1)
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.plot(train_losses)
plt.grid()

plt.subplot(1,2,2)
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.plot(train_accs, label = 'train')
plt.plot(test_accs, label = 'test')
plt.legend()
plt.grid()

Visualize results with Tensorboard:

In [ ]:
# Open Tensorboard
%tensorboard --logdir runs/